## This file replaces the D-MPNN representation with RDKit representation

In [1]:
import numpy as np
import json
import pandas as pd

In [2]:
len_unirep = 1900
len_dmpnn_rep = 300

In [3]:
with open('../RDKit_reps.json','r') as f:
    aa_reps = json.loads(f.read())
    for key, val in aa_reps.items():
        lst = aa_reps[key]
        lst = [float(x) for x in lst]
        lst = np.array(lst)
        lst = lst.reshape((-1,1))
        aa_reps[key] = lst
    len_rdkit_rep = len(aa_reps['M'])
    
aa_reps_one_hot = {}
num_aa = len(aa_reps)
for key,val in enumerate(aa_reps):
    one_hot = np.zeros((num_aa,1))
    one_hot[key] = 1.
    aa_reps_one_hot[val] = one_hot

In [4]:
def get_aa_encoding_neighbor(seq, damp_param):
    lst_seq = list(seq)
    rep = aa_reps_one_hot[lst_seq[0]]*damp_param+aa_reps_one_hot[lst_seq[1]]
    if len(lst_seq)>2:
        rep =  aa_reps_one_hot[lst_seq[0]]*damp_param+aa_reps_one_hot[lst_seq[1]]+aa_reps_one_hot[lst_seq[2]]*damp_param
    else:
        rep =  aa_reps_one_hot[lst_seq[0]]*damp_param+aa_reps_one_hot[lst_seq[1]]
    return rep 

In [5]:
hs64_mat = np.load('hs1900_nl1_0_to_56097.npy')
hs64_mat = np.delete(hs64_mat, np.arange(len_unirep,len_unirep+len_dmpnn_rep), axis=0)
hs64_mat = np.insert(hs64_mat, len_unirep, np.zeros((num_aa, hs64_mat.shape[1])), axis=0)
hs64_mat.shape

(1927, 56064)

In [6]:
df_active_site = pd.read_excel('active_site_annotation_neighbor.xlsx',header=[0],index_col=[0])
df_active_site.head()

,Entry,Entry name,Organism,Length,Active site num,Active site type,Active site residue,Sequence,UniRep,Active site neighbor
0,O35815,ATX3_RAT,Rattus norvegicus (Rat),355,14,2,C,MESIFHEKQEGSLCAQHCLNNLLQGEYFSPVELSSIAHQLDEEERL...,NaN,LCA
1,O35815,ATX3_RAT,Rattus norvegicus (Rat),355,119,5,H,MESIFHEKQEGSLCAQHCLNNLLQGEYFSPVELSSIAHQLDEEERL...,NaN,EHW
2,O94760,DDAH1_HUMAN,Homo sapiens (Human),285,173,4,H,MAGLGHPAAFGRATHAVVRALPESLGQHALRSAKGEEVDVARAERQ...,NaN,LHL
3,O94760,DDAH1_HUMAN,Homo sapiens (Human),285,274,2,C,MAGLGHPAAFGRATHAVVRALPESLGQHALRSAKGEEVDVARAERQ...,NaN,TCC
4,Q540U1,APHA_SALTM,Salmonella typhimurium,237,69,2,D,MKKITLALSAVCLLFTLNHSANALVSSPSTLNPGTNVAKLAEQAPV...,NaN,FDI


In [7]:
for row in range(hs64_mat.shape[1]):
    res_name = df_active_site.iloc[row]['Active site neighbor']
    act_site_type_df = int(df_active_site.iloc[row]['Active site type'])-1
    act_site_type_mat = int(hs64_mat[num_aa+ len_unirep,row])
#     print(act_site_type_df, act_site_type_mat)
    assert act_site_type_df == act_site_type_mat
    hs64_mat[len_unirep:len_unirep+num_aa,row:row+1] = get_aa_encoding_neighbor(res_name,0.1)


In [8]:
np.save('hs1900_nl1_onehotneigh_0_to_56097.npy', hs64_mat)

In [9]:
orig_mat = np.load('hs1900_nl1_0_to_56097.npy')
new_mat = np.load('hs1900_nl1_onehotneigh_0_to_56097.npy')


In [10]:
np.max(np.abs(orig_mat[-1,:]-new_mat[-1,:] ))

0.0

In [11]:
np.max(np.abs(orig_mat[:len_unirep,:]-new_mat[:len_unirep,:] ))

0.0

In [12]:
new_mat.shape

(1927, 56064)

In [13]:
len(aa_reps)

26